In [24]:
from pulp import *
import os
import numpy as np
import pandas as pd
import gurobipy as grb

def read_matrix(path_file):
    satisfactions = []
    resources = []
    capacities = []
    
    with open(path_file, 'r') as f:
        m, n= f.readline().split(' ')[:2]
        m, n = int(m), int(n)
        lines = f.readlines()
        lines = [line.rstrip().split(' ') for line in lines]

    
        for i in range(0, m):
            for j in range(n):
                satisfactions.append(int(lines[i][j]))


        for i in range(m, m + m):
            for j in range(n):
                resources.append(int(lines[i][j]))
        
        for j in range(m):
            capacities.append(int(lines[2*m][j]))
            
    satisfactions = np.array(satisfactions)
    resources = np.array(resources)
    capacities = np.array(capacities)
    
    return m, n, satisfactions, resources, capacities

In [11]:
# References: 
# How to keep log files: https://coin-or.github.io/pulp/guides/how_to_configure_solvers.html

In [66]:
inputPath = "./input/e801600.in"

In [67]:
ans = []

prob = LpProblem("desig_gen_problem", LpMaximize)

m, n, sat, res, cap = read_matrix(inputPath)

N = range(n*m)

x = LpVariable.dicts(
    "x", N, lowBound=0, upBound=1.0
)

prob += lpSum([sat[i] * x[i]  for i in N]), 'objective F'

#   every job can only be picked by one worker
for i in range(n):
    prob += lpSum([x[i + j*n] for j in range(m)]) == 1

#   every worker has to take only the jobs that they can
for i in range(m):
    prob += lpSum([x[i*n + j] * res[i*n + j] for j in range(n)]) <= cap[i]
file = prob.writeMPS("test.mps")
# result = prob.solve(solver)
# solver.Attr

In [68]:
model = grb.read("test.mps")
model.setParam("TimeLimit", 100.0)
model.setParam("Presolve",0)
model.setParam("Cuts",0)
model.setParam("BranchDir",-1)

Read MPS format model from file test.mps
Reading time = 0.53 seconds
desig_gen_problem: 1680 rows, 128000 columns, 256000 nonzeros
Set parameter TimeLimit to value 100
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter BranchDir to value -1


In [69]:
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1680 rows, 128000 columns and 256000 nonzeros
Model fingerprint: 0x7070c135
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.09 seconds (0.00 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 117 iterations and 0.11 seconds (0.00 work units)
Optimal objective -1.597168000e+06


In [70]:
# numero de nos explorados
model.getAttr("NodeCount")

0.0

In [74]:
# valor da func objetivo
model.getAttr("ObjVal")

-1597168.0

In [79]:
# limitante dual
model.getAttr("Pi") 

(1680,)